# 1_Instalation de Pyspark

In [162]:
#Install Pyspark
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2_Import des librairies

In [163]:
import findspark
findspark.init
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import StructType,StringType,IntegerType,MapType,ArrayType,StructField
from pyspark.sql.functions import count,when,col,isnan,from_json,explode,map_keys,udf,coalesce,lit,to_date,year,split,get_json_object,array,collect_list,concat_ws,regexp_replace,regexp_extract
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import plotly.graph_objects as go
import plotly.express as px
from wordcloud import WordCloud
import os



# 3_Spark session

In [164]:
#Start Spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

# 4_Import dataset

In [165]:
#connexion to my drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [166]:
#import datasets
ratings=spark.read\
.format("com.databricks.spark.csv")\
.option("multiline",True)\
.option("header",True)\
.option("escape", "\"")\
.option("inferschema",True)\
.csv("/content/drive/MyDrive/archive/ratings.csv")

# 5_Systeme de recommandation en utilisant ALS 

**Intêret d'utiliser le modele ALS :**

Nous avons choisi d'utiuliser le modele ALS pour les raiusons suivantes :      

Le modèle ALS (Alternating Least Squares) est utilisé pour résoudre des problèmes de factorisation de matrices, comme la recommandation de films ou de produits. Il fonctionne en décomposant une matrice en deux matrices plus petites, appelées matrices de facteurs, qui contiennent des informations sur les utilisateurs et les articles. Ces matrices de facteurs sont ensuite utilisées pour prédire les notes ou les évaluations manquantes dans la matrice d'origine.

En utilisant PySpark, nous pouvons paralléliser le calcul du modèle ALS sur plusieurs machines, ce qui permet de traiter des ensembles de données plus volumineux et de réduire les temps de calcul. Cela rend le modèle ALS plus efficace pour les grandes entreprises qui ont des besoins de traitement de données massives.

In [167]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

print("Un aperçu du dataframe ratings : ")
ratings.select('userId','movieId','rating','timestamp').show(5)

#Creation du train et du test avec respectivement 80% et 20% du df original ratings
(training, test) = ratings.randomSplit([0.8, 0.2])


als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

model = als.fit(training)

# Evaluatation du modele en utilsiant la RMSE sur les données test
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Un aperçu du dataframe ratings : 
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    110|   1.0|1425941529|
|     1|    147|   4.5|1425942435|
|     1|    858|   5.0|1425941523|
|     1|   1221|   5.0|1425941546|
|     1|   1246|   5.0|1425941556|
+------+-------+------+----------+
only showing top 5 rows

Root-mean-square error = 0.8427603174530015


In [168]:
# Top 10 des films recommandés pour chaque utilisateur
userRecs = model.recommendForAllUsers(10)

# Top 10 des utilisateurs suceptibes d'appréciés chaque films
movieRecs = model.recommendForAllItems(10)


In [160]:
# On concertit le dataframe spark en dataframe pandas pour requeter les utilisateurs
userRecs_df = userRecs.toPandas()
movieRecs_df = movieRecs.toPandas()

In [161]:
def get_recommended_film_for_given_user(user_id,n):
    line = userRecs_df[userRecs_df['userId'] == user_id]['recommendations']
    df = pd.DataFrame(line.tolist(), columns=["1", "2","3","4","5","6","7","8","9","10"])
    df = df.T
    df = df.rename(columns={0: "Predictions"})
    print(f"Pour l'utilisateur {user_id}, le top {n} des films à voir par ordre de priorité sont les suivants (chaque film a    une note associée :")
    return df.head(n)

In [169]:
get_recommended_film_for_given_user(65,10)

Pour l'utilisateur 65, le top 10 des films à voir par ordre de priorité sont les suivants (chaque film a    une note associée :


,Predictions
1,"(174817, 12.147197723388672)"
2,"(150667, 11.896146774291992)"
3,"(153022, 9.8738374710083)"
4,"(137112, 9.612909317016602)"
5,"(170731, 9.289995193481445)"
6,"(174799, 9.24531078338623)"
7,"(116387, 9.17775821685791)"
8,"(78329, 9.131409645080566)"
9,"(173283, 8.82858943939209)"
10,"(154882, 8.776744842529297)"


In [170]:
get_recommended_film_for_given_user(5678,10)

Pour l'utilisateur 5678, le top 10 des films à voir par ordre de priorité sont les suivants (chaque film a    une note associée :


,Predictions
1,"(150667, 17.45391273498535)"
2,"(170731, 15.960968017578125)"
3,"(152711, 15.3615083694458)"
4,"(170477, 12.068924903869629)"
5,"(127126, 11.878308296203613)"
6,"(96255, 11.138431549072266)"
7,"(161292, 10.771849632263184)"
8,"(142092, 10.658073425292969)"
9,"(100902, 10.491951942443848)"
10,"(118286, 10.3538818359375)"


In [171]:
get_recommended_film_for_given_user(5,5)

Pour l'utilisateur 5, le top 5 des films à voir par ordre de priorité sont les suivants (chaque film a    une note associée :


,Predictions
1,"(174817, 13.085582733154297)"
2,"(137112, 12.931806564331055)"
3,"(165367, 12.351509094238281)"
4,"(74406, 12.324579238891602)"
5,"(150667, 12.089497566223145)"


Bonus : Suggestion des utilisateurs suceptibles d'apprecier un film

In [172]:
def get_recommended_user_for_a_given_film(movieId,n):
    line = movieRecs_df[movieRecs_df['movieId'] == movieId]['recommendations']
    df = pd.DataFrame(line.tolist(), columns=["1", "2","3","4","5","6","7","8","9","10"])
    df = df.T
    df = df.rename(columns={0: "Predictions"})
    print(f"Pour le film {movieId}, le top {n} des utilisateurs qui seraient suceptibles d'aimer le film par ordre de priorité sont les suivants :")
    return df.head(n)

In [173]:
get_recommended_user_for_a_given_film(678,10)

Pour le film 678, le top 10 des utilisateurs qui seraient suceptibles d'aimer le film par ordre de priorité sont les suivants :


,Predictions
1,"(232405, 6.303138256072998)"
2,"(31289, 6.2949934005737305)"
3,"(9364, 6.224935531616211)"
4,"(140403, 6.156093120574951)"
5,"(219866, 6.14460563659668)"
6,"(119679, 6.112834453582764)"
7,"(229270, 6.1019487380981445)"
8,"(236676, 6.086291313171387)"
9,"(44195, 6.061352729797363)"
10,"(145743, 6.0503129959106445)"
